# Download the necessary libraries

In [ ]:
!pip uninstall telebot
!pip uninstall PyTelegramBotAPI

In [ ]:
!pip install pyTelegramBotAPI
!pip install --upgrade pyTelegramBotAPI

!pip install silero
!pip install transformers
!pip install pytorch_transformers

In [4]:
import torch
import telebot
import subprocess
import requests
from glob import glob
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
description = ['It message is negative', 'It message is positive, dont worry']

In [5]:
# load tokenizer
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", do_lower_case=False)

In [ ]:
# load model
from pytorch_transformers import BertForSequenceClassification


bert_model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
bert_model.load_state_dict(torch.load('/content/drive/MyDrive/NLP/weights_for_prediction.h5', map_location=torch.device('cpu')), strict=False)

bert_model.to(device)

bert_model.eval() #freeze BatchNorm layers

In [7]:
# load audio model

au_model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device='cpu')

(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/snakers4/silero-models/zipball/master" to /root/.cache/torch/hub/master.zip


  0%|          | 0.00/112M [00:00<?, ?B/s]

In [8]:
def read_file(file_name, mode):
    with open(file_name, mode) as file:
        return file.read()


def audio_to_text(path_to_audio, dest_filename='outputs.wav'):

    device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU
    decode_message = ''

    process = subprocess.run(['ffmpeg', '-i', path_to_audio, dest_filename])
    if process.returncode != 0:
        raise Exception("Something went wrong")


    test_files = glob(dest_filename)
    batches = split_into_batches(test_files, batch_size=10)
    input = prepare_model_input(read_batch(batches[0]),
                                device=device)

    output = au_model(input)

    for sample in output:
        decode_message += decoder(sample.cpu())

    os.remove(path_to_audio)
    os.remove(dest_filename)

    return decode_message


def predict(model, text, device='cpu'):
    with torch.no_grad():
        if torch.cuda.is_available():
            inputs = tokenizer(text, padding=True, truncation=True, max_length=256, return_tensors="pt").to(device)
            inputs = inputs.to(device)

        inputs = tokenizer(text, padding=True, truncation=True, max_length=256, return_tensors="pt").to(device)

        outputs = model(**inputs)
        probs = outputs[0].softmax(1)
        idx = probs.argmax().item()
    return description[idx]

# Creating a telegram bot

In [9]:
token = read_file('token_telebot.txt', 'r') #token is a secret information, save it in file
bot = telebot.TeleBot(token)  

In [11]:
@bot.message_handler(content_types=['text'])
def get_user_message(message):
    bot.send_message(message.chat.id,
                    f"Oh hey! I'm a bot that can tell a message good or bad before you listen to it")


@bot.message_handler(content_types=['voice'])
def get_user_audio(message):
    bot.reply_to(message, 'The audio is being processed...')
    try:
        file_info = bot.get_file(message.voice.file_id)
        file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(token, file_info.file_path))

        with open('voice.ogg', 'wb') as f:
            f.write(file.content)

        text = audio_to_text('/content/voice.ogg')

        sentiment = predict(bert_model, text)
        bot.send_message(message.from_user.id, format(sentiment))



    except Exception as e:
        bot.reply_to(message, e)


bot.polling(none_stop=True)